In [1]:
# import packages to read sqlite file
import sqlite3
import pandas as pd
import time  

# import packages to write a json file
import os
import json

- Getting Unix timestamps in PTG format.
- Getting artifical timestamps that was generated given a ```first-entry``` and ```last-entry``` timestamps in PTG format.

In [2]:
# TimeStamps

# generate artificial timestamps
def preprocessTimestampData(recordingMetaData, count_row):
    result = []
    rawFirstEntryTimestamp = recordingMetaData['first-entry'].split('-')[0]
    rawLastEntryTimestamp = recordingMetaData["last-entry"].split('-')[0]
    duration = int(rawLastEntryTimestamp) - int(rawFirstEntryTimestamp); # get total duration from json files.
    intervals = duration/count_row
    for playedTime in range(count_row):
        currentTime = int(playedTime * intervals)
        result.append(str(int(rawFirstEntryTimestamp) + currentTime) + "-0")
    return result

# get Unix Timestamp in PTG format
def get_timeStamp():
    unix_timestamp = time.time()  # Unix Timestamp: Applying time() method             
    unix_timestamp_ptg = str(int(unix_timestamp*1000))+"-0"  # convert to PTG timestamp format
    return unix_timestamp_ptg

Create a database connection to the SQLite database and return this as a dataframe

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file, uri=True)
    except Error as e:
        print(e)
    return conn

def get_dataframe_from_sqlite(db_file, table_name):
    # fancy read-only connection
    conn = create_connection(db_file)
    # create the dataframe from a query
    query = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query, conn)
    return df

#### Getting actions from a dataframe

In [3]:


def get_actions_json(df, artificialTimeStamps=None):
    json_values = []
    unique_actions = df["Event"].unique()

    for index, row in df.iterrows():
        row_action = row['Event']
        # row_dic = {"timestamp": row['timestamp']} # get time from sqlite file
#         row_dic = {"timestamp": get_timeStamp()}    # get current time
        if(artificialTimeStamps):
            row_dic = {"timestamp": artificialTimeStamps[index]}
        else:
            row_dic = {"timestamp": row['timestamp']} # get time from sqlite file
        for action in unique_actions:
            if (action == row_action):
                row_dic[action] = 1
            else:
                row_dic[action] = 0
        json_values.append(row_dic)
    return json_values



#### Getting steps from a dataframe


In [17]:
# {
#   "step_id": 0,
#   "step_status": "NEW",
#   "step_description": "Place tortilla on cutting board.",
#   "error_status": false,
#   "error_description": "",
#   "timestamp": "1679339146083-0"
# }

def get_steps_json(df, artificialTimeStamps=None, substeps=False):
    json_values = []

    for index, row in df.iterrows():
        row_step = row['Step'] if row['Step'] != None else "None" 
        step = row_step if substeps else row_step.split('.')[0] # substeps
        # row['timestamp'] # get time from sqlite file
        timestamp_value = artificialTimeStamps[index] if artificialTimeStamps else row['timestamp']

        row_dic = {"step_id": step,
                "step_status": "NEW",
                "step_description": "",
                "error_status": False,
                "error_description": "",
                "timestamp": timestamp_value
               }
        json_values.append(row_dic)
    return json_values



### Getting actions from a SQLite database.
An artificial timestamps is used, where the ``first-entry`` is ``"1683929511993-0"`` and  ``last-entry`` is ``"1683929525968-0"``.


In [5]:
filename = 'egovlp_action_steps_v4.json' # 'detic-image-fixed-labels.json'
isFile = os.path.isfile(filename)

df_table = get_dataframe_from_sqlite("0293_13.sqlite", "ocarina_mission_log")

# Generate artificial timestamps
count_row = df_table.shape[0]  # Gives number of rows
recordingMetaData = {'first-entry': "1683929511993-0", "last-entry": "1683929525968-0"}
artificialTimeStamps = preprocessTimestampData(recordingMetaData, count_row)


json_data = get_actions_json(df_table, artificialTimeStamps)

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 

### Getting steps from a SQLite database.
An artificial timestamps is used, where the ``first-entry`` is ``"1683929511993-0"`` and  ``last-entry`` is ``"1683929525968-0"``.

In [18]:
filename = 'reasoning_check_status_substeps1.json' # 'detic-image-fixed-labels.json'
isFile = os.path.isfile(filename)

df_table = get_dataframe_from_sqlite("0293_13.sqlite", "ocarina_mission_log")

# Generate artificial timestamps
count_row = df_table.shape[0]  # Gives number of rows
recordingMetaData = {'first-entry': "1683929511993-0", "last-entry": "1683929525968-0"}
artificialTimeStamps = preprocessTimestampData(recordingMetaData, count_row)


json_data = get_steps_json(df_table, artificialTimeStamps)

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 